<img style="float: left; margin:0px 15px 15px 0px; width:120px" src="https://www.orfeo-toolbox.org/wp-content/uploads/2016/03/logo-orfeo-toolbox.png">

# OTB Guided Tour - JURSE 2019 Vannes France - May 21th
## Yannick TANGUY and David YOUSSEFI (CNES, French Space Agency)

<br>

<b> Press <span style="color:black;background:yellow">SHIFT+ENTER</span> to execute the notebook interactively cell by cell </b></div>


## Step 2 : 

The aim of this second exercise is to compute a water mask, from NDWI2 values
- what kind of function do we need to implement ?
- are all the pixels valid ?

In [ ]:
def app_radiometricindices(images):
    index = 0
    for im in images:
        # Here we create an application
        app = otbApplication.Registry.CreateApplication("BandMath")
        # BandMath takes a list of images as input,
        # so we have to give a Python list [ image ], or [ image1, image2, .., imageN]
        app.SetParameterStringList("il",[im])
        app.SetParameterString("out",im.replace("RVBPIR.tif","_ndwi2.tif"))
        app.SetParameterString("exp","(im1b2-im1b4)/(im1b2+im1b4)")
        app.Execute()
        if (index == 0):
            tab_App = [app]
        else:
            tab_App.append(app)
        index += 1
    return tab_App